# Recording Events
AgentOps has a number of different [Event Types](https://docs.agentops.ai/v1/details/events)

AgentOps automatically instruments your LLM Calls from OpenAI, LiteLLM, and Cohere. Just make sure their SDKs are imported before initializing AgentOps like we see below

In [ ]:
import agentops
import openai

# Create new session
agentops.init()

# Optionally, we can add tags to the session
# agentops.init(tags=['Hello Tracker'])

message = ({"role": "user", "content": "Hello"},)
response = openai.chat.completions.create(
    model="gpt-3.5-turbo", messages=message, temperature=0.5
)

Click the AgentOps link above to see your session!

## Action Event

AgentOps allows you to record other actions. The easiest way to record actions is through the use of AgentOp's decorators

In [ ]:
from agentops import record_function


@record_function("add numbers")
def add(x, y):
    return x + y


add(2, 4)

We can also manually craft an event exactly the way we want by creating and recording an `ActionEvent`

In [ ]:
from agentops import ActionEvent

agentops.record(
    ActionEvent(
        action_type="Agent says hello", params={"message": "Hi"}, returns="Hi Back!"
    )
)

## Tool Event
Agents use tools. These tools are useful to track with information such as name, end status, runtime, etc. To record tool usage, you can create and record a `ToolEvent` similar to above.

In [ ]:
from agentops import ToolEvent, record


def scrape_website(url: str):
    tool_event = ToolEvent(
        name="scrape_website", params={"url": url}
    )  # the start timestamp is set when the obj is created
    result = integration.scrape_website(data)  # perform tool logic
    tool_event.returns = result
    record(tool_event)

## Error Events
Error events can be used alone or in reference to another event. Lets add a catch block to the code above

In [ ]:
from agentops import ToolEvent, record, ErrorEvent


def scrape_website(url: str):
    tool_event = ToolEvent(
        name="scrape_website", params={"url": url}
    )  # the start timestamp is set when the obj is created

    try:
        1 / 0  # Ooops! Something went wrong
    except Exception as e:
        record(ErrorEvent(exception=e, trigger_event=tool_event))


scrape_website("https://app.agentops.ai")

agentops.end_session("Success")